# Tutorial 8 - Solver options

在教程7中，我们了解了如何更改模型选项。在本教程中，我们将演示如何将选项传递给解算器。

Pybam中的所有模型都有一个默认解算器，该解算器通常不同，这取决于模型是生成常微分方程（ODE）还是微分代数方程（DAE）系统。

要更改的最常见选项之一是解算器公差。默认情况下，所有公差都设置为$10^{-6}$。但是，根据您的模拟，您可能会发现需要收紧公差以获得更精确的解决方案，或者需要松开公差以减少求解时间。进行公差研究是一种很好的做法，您可以用更严格的公差模拟相同的问题，并比较结果。我们在这里不展示如何做到这一点，但在下一个教程中，我们将以类似的方式给出一个网格分辨率研究的示例。

In [3]:
import pybamm

在这里，我们将更改绝对和相对公差，以及CasadiSolver的“模式”。有关所有解算器选项的列表，请参阅文档。

CasadiSolver可以在多种模式下运行，包括“安全”（默认）和“快速”。安全模式执行步进和检查集成，并支持事件处理（例如，您可以集成，直到达到特定电压），建议用于模拟完全充电或放电。快速模式执行直接积分，忽略事件，建议在模拟驾驶循环或其他不应触发事件的模拟时使用。

我们将在“安全”和“快速”模式下使用所有默认选项解决DFN，并比较解决方案。对于这两种模拟，我们将使用$10^{-3}$作为绝对和相对公差。出于演示目的，我们将切断电压更改为3.6V，以便观察两种解算器模式的不同行为。

In [4]:
# load model and parameters
model = pybamm.lithium_ion.DFN()
param = model.default_parameter_values
param["Lower voltage cut-off [V]"] = 3.6

# load solvers
safe_solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="safe")
fast_solver = pybamm.CasadiSolver(atol=1e-3, rtol=1e-3, mode="fast")

# create simulations
safe_sim = pybamm.Simulation(model, parameter_values=param, solver=safe_solver)
fast_sim = pybamm.Simulation(model, parameter_values=param, solver=fast_solver)

# solve
safe_sim.solve([0, 3600])
print("Safe mode solve time: {}".format(safe_sim.solution.solve_time))
fast_sim.solve([0, 3600])
print("Fast mode solve time: {}".format(fast_sim.solution.solve_time))

# plot solutions
pybamm.dynamic_plot([safe_sim, fast_sim])

Safe mode solve time: 580.623 ms
Fast mode solve time: 132.312 ms


interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

我们看到，两个解算器在达到截止电压时给出相同的解。此时，使用“安全”模式的解算器停止，但使用“快速”模式的解算器将继续积分，直到最后一次。顾名思义，“快速”模式比“安全”模式集成得更快，但如果您的模拟需要处理事件，则该模式不适用。

通常，默认解算器选项提供了速度和精度的良好组合，但我们鼓励您研究不同的解算器和选项，以找到问题的最佳组合。

在下一个教程中，我们将演示如何更改网格。